In [1]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2018,7)

In [2]:
input_text="""Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin."""
input_lines = input_text.splitlines()

In [13]:
import re
pattern_input = re.compile(r'Step (\w).*step (\w).*')

def parse_input(lines):
    requirements = {}
    for line in lines:
        match = pattern_input.match(line)
        if match:
            step_a = match.group(1)
            step_b = match.group(2)
            if step_b not in requirements:
                requirements[step_b] = {}
                requirements[step_b]['prereqs'] = []
                requirements[step_b]['next'] = []
            if step_a not in requirements:
                requirements[step_a] = {}
                requirements[step_a]['prereqs'] = []
                requirements[step_a]['next'] = []
            requirements[step_b]['prereqs'].append(step_a)
            requirements[step_a]['next'].append(step_b)
    return requirements

def dependencies_satisfied(instructions, step, result):
    ready = True
    dependencies = instructions[step]['prereqs']
    for prev_step in dependencies:
        if prev_step not in result:
            ready = False
    return ready

def get_order(instructions):
    result = ''
    available = []
    while len(result) < len(instructions.keys()):
        if len(available) > 0:
            available.sort()
            result += available.pop(0)
        for step, instruction in instructions.items():
            # skip if we have already placed the step
            if step in result:
                continue
            if dependencies_satisfied(instructions, step, result):
                if step not in list(result) + available:
                    available.append(step)
                for next_step in sorted(instruction['next']):
                    if dependencies_satisfied(instructions, next_step, result):
                        if next_step not in list(result) + available:
                            available.append(next_step)
    return result


def time_per_step(step, seconds):
    return seconds + ord(step) - ord('A')

def build(instructions, order, worker_count=4, seconds=60):
    # convert order to list
    order = list(order)
    in_progress = []
    completed = ''
    # build workers:
    workers = []
    for _ in range(worker_count):
        workers.append({"step": None, "time_left": 0})
    second = -1
    while len(completed) < len(instructions.keys()):
        second += 1
        for worker in workers:
            #print(worker)
            # is worker idle
            if worker['time_left'] == 0:
                if worker['step']:
                    #print(f"{worker['step']} completed")
                    completed += worker['step']
                    in_progress.pop(in_progress.index(worker['step']))
                    worker['step'] = None
                # find new step to work on
                for step in order:
                    if step not in list(completed) + in_progress and dependencies_satisfied(instructions, step, completed):
                        worker['step'] = step
                        worker['time_left'] = time_per_step(step, seconds)
                        in_progress.append(step)
                        #print(f"{worker['step']} started")
                        break
            else:
                worker['time_left'] -= 1
                #print(f"{worker['step']} time left {worker['time_left']}")
        #print(f"{second} {' '.join(str(worker['step']) for worker in workers)} {completed}")
    return second
            
reqs = parse_input(input_lines)
print(reqs)
order = get_order(reqs)
print(order)
seconds = build(reqs, order, 2, 0)
print(seconds)

{'A': {'prereqs': ['C'], 'next': ['B', 'D']}, 'C': {'prereqs': [], 'next': ['A', 'F']}, 'F': {'prereqs': ['C'], 'next': ['E']}, 'B': {'prereqs': ['A'], 'next': ['E']}, 'D': {'prereqs': ['A'], 'next': ['E']}, 'E': {'prereqs': ['B', 'D', 'F'], 'next': []}}
CABDFE
15


In [4]:
result = 'abc'
available = ['d']
for step in 'abcde':
    print(step, step not in list(result) + available)

a False
b False
c False
d False
e True


In [5]:
time_per_step('C', 0)

3

In [6]:
workers = [{"step": 'A'}, {"step": None}, {"step": 'C'}]
' '.join(str(worker['step']) for worker in workers)

'A None C'